# Exploring s1ard data cubes

## Introduction

This example notebook will give a short demonstration of how S1-NRB products can be explored as on-the-fly data cubes with little effort by utilizing the STAC metadata provided with each product. It is not intended to demonstrate how to process the S1-NRB products in the first place. For this information please refer to the  [usage instructions](https://s1ard.readthedocs.io/en/latest/general/usage.html).

A lightning talk related to this topic has been given during the [Cloud-Native Geospatial Outreach Event 2022](https://medium.com/radiant-earth-insights/cloud-native-geospatial-outreach-2022-recap-and-videos-2a0e80085db7), which can be found [here](https://www.youtube.com/watch?v=NOwjDvB9-0k).

Follow [this link](https://nbviewer.org/github/SAR-ARD/s1ard/blob/main/docs/examples/nrb_cube.ipynb) for a better visualization of this notebook!

**Sentinel-1 Normalised Radar Backscatter**
Sentinel-1 Normalised Radar Backscatter (S1-NRB) is a newly developed Analysis Ready Data (ARD) product for the European Space Agency that offers high-quality, radiometrically terrain corrected (RTC) Synthetic Aperture Radar (SAR) backscatter and is designed to be compliant with the CEOS ARD for Land (CARD4L) [NRB specification](https://ceos.org/ard/files/PFS/NRB/v5.5/CARD4L-PFS_NRB_v5.5.pdf).
You can find more detailed information about the S1-NRB product [here](https://sentinel.esa.int/web/sentinel/sentinel-1-ard-normalised-radar-backscatter-nrb-product).

**SpatioTemporal Asset Catalog (STAC)**
All S1-NRB products include metadata in JSON format compliant with the [SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/) specification.
STAC uses several sub-specifications ([Item](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md), [Collection](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md) & [Catalog](https://github.com/radiantearth/stac-spec/blob/master/catalog-spec/catalog-spec.md)) to create a hierarchical structure that enables efficient querying and access of large volumes of geospatial data.

## Getting started

After following the [installation instructions](https://s1ard.readthedocs.io/en/latest/general/installation.html) you need to install an additional package into the activated conda environment:

```bash
conda activate s1ard   
conda install stackstac
```

Let's assume you have a collection of S1-NRB scenes located on your local disk, a fileserver or somewhere in the cloud. As mentioned in the [Introduction](#Introduction), each S1-NRB scene includes metadata as a STAC Item, describing the scene's temporal, spatial and product specific properties.

The **only step necessary to get started** with analysing your collection of scenes, is the creation of STAC Collection and Catalog files, which connect individual STAC Items and thereby create a hierarchy of STAC objects. `s1ard` includes the utility function [make_catalog](https://s1ard.readthedocs.io/en/latest/api.html#s1ard.metadata.stac.make_catalog), which will create these files for you. Please note that `make_catalog` expects a directory structure based on MGRS tile IDs, which allows for efficient data querying and access. After user confirmation it will take care of reorganizing your S1-NRB scenes if this directory structure doesn't exist yet.

In [3]:
import numpy as np
import stackstac
from s1ard.metadata.stac import make_catalog

nrb_catalog = make_catalog(directory='./NRB_thuringia', product_type='NRB', silent=True)

   
The STAC Catalog can then be used with libraries such as [stackstac](https://github.com/gjoseph92/stackstac), which _"turns a STAC Collection into a lazy xarray.DataArray, backed by dask"._ 

The term _lazy_ describes a method of execution that only computes results when actually needed and thereby enables computations on larger-than-memory datasets. _[xarray](https://xarray.pydata.org/en/stable/index.html)_ is a Python library for working with labeled multi-dimensional arrays of data, while the Python library _[dask](https://docs.dask.org/en/latest/)_ facilitates parallel computing in a flexible way.

Compatibility with [odc-stac](https://github.com/opendatacube/odc-stac), a very [similar library](https://github.com/opendatacube/odc-stac/issues/54) to stackstac, has also been implemented.

In [4]:
aoi = (10.638066, 50.708415, 11.686751, 50.975775)
ds = stackstac.stack(items=nrb_catalog, bounds_latlon=aoi, 
                     dtype=np.dtype('float32'), chunksize=(-1, 1, 1024, 1024))
ds

As you can see in the output above, the collection of S1-NRB scenes was successfully loaded as an `xarray.DataArray`. The metadata attributes included in all STAC Items are now available as coordinate arrays (see [here](https://docs.xarray.dev/en/stable/user-guide/terminology.html#term-Coordinate) for clarification of Xarray's terminology) and can be utilized during analysis.

It is now possible to explore and analyse the S1-NRB data cube. The most important tools in this regard are the already mentioned xarray and dask. Both are widely used and a lot of tutorials and videos can be found online, e.g. in the xarray Docs ([1](https://docs.xarray.dev/en/stable/user-guide/index.html), [2](https://docs.xarray.dev/en/stable/tutorials-and-videos.html)) or the [Pangeo Tutorial Gallery](https://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/index.html).